# 2.Pobieranie Apache Spark i ropoczęcie pracy

In [4]:
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType, MapType, DateType, TimestampType
from pyspark.sql.functions import (
    col, size, lit, explode, 
    concat, concat_ws, substring, 
    datediff, date_add, date_sub,
    year, month, dayofmonth, dayofweek, dayofyear, weekofyear, 
    hour, minute, second,
    count, min, max, avg, sum, udf, when
)
from datetime import datetime

spark = SparkSession.builder.appName("spark").getOrCreate()

In [6]:
strings = spark.read.text("data/wikipedia-iso-country-codes.csv")
strings.show(10, truncate=False)

+-------------------------------------------------------------------------------+
|value                                                                          |
+-------------------------------------------------------------------------------+
|English short name lower case,Alpha-2 code,Alpha-3 code,Numeric code,ISO 3166-2|
|Afghanistan,AF,AFG,004,ISO 3166-2:AF                                           |
|Åland Islands,AX,ALA,248,ISO 3166-2:AX                                         |
|Albania,AL,ALB,008,ISO 3166-2:AL                                               |
|Algeria,DZ,DZA,012,ISO 3166-2:DZ                                               |
|American Samoa,AS,ASM,016,ISO 3166-2:AS                                        |
|Andorra,AD,AND,020,ISO 3166-2:AD                                               |
|Angola,AO,AGO,024,ISO 3166-2:AO                                                |
|Anguilla,AI,AIA,660,ISO 3166-2:AI                                              |
|Antarctica,AQ,A